# News headline topic analysis with Latent Dirichlet allocation (LDA)
The purpose of this analysis is to find dominant topics across news headlines (25 per day, over 1 year). These topics will later be correlated with daily stock market loss/gain information to understand how certain topics influence the stock market.

## What is LDA?
Latent Dirichlet allocation is a way to create groupings of higher-level topics out of individual words present in any set of document (in our case, news headlines). You can use LDA to get a sense of the overall themes in a set of documents. 

LDA is an unsupervised machine learning model that works by analyzing two things: a distribution of topics in a document, and a distribution of words in a topic. The "topics" it finds aren't specific words (e.g., "This headline is about 'war'") but instead conceptually similar groups of words that together make up a theme (e.g., "This headline is similar to the words 'war', 'crisis', 'iran'...").

Once you've created your LDA model, you can feed in a document and the model will score the overall relevancy of your document against the main _x_ topics found in your overall corpus. In other words, it will tell you which of the main topics found in the overall corpus are also found in your document, and to what extent.

## Why LDA?
LDA is a fairly popular topic modelling choice among NLP professionals, and relatively straightforward to implement. Using LDA, we were able to find 10 dominant themes in eight years' worth of news headlines within about an hour—a task that would take a human days to analyze.

## How does text pre-processing affect the accuracy of LDA?
We tested two pre-processing approaches: Bag of Words and TF-IDF, each of which count how often words appear in a document. The TF-IDF model tries to understand which words are more important than others in a document, and scores them accordingly. Presumably, TF-IDF preprocessing will help train a more accurate LDA model.

## How does LDA perform compared to other topic models?
More here later.

## Credit
Parts of this work borrow from excellent Susan Li's ["Topic Modeling and LDA in Python"](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24) tutorial.

# Load the data from CSV, dependencies

In [1]:
# import dependencies
import pandas as pd

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import numpy as np
np.random.seed(22)

[nltk_data] Downloading package wordnet to /Users/stacy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Import the data
data = pd.read_csv("../Data/RedditNews.csv")

In [3]:
data.head()

# 73,608 records from 2016-2018

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...


In [4]:
# Getting just the headlines for our corpus
headlines = data[['News']]
headlines.head()

,News
0,A 117-year-old woman in Mexico City finally re...
1,IMF chief backs Athens as permanent Olympic host
2,"The president of France says if Brexit won, so..."
3,British Man Who Must Give Police 24 Hours' Not...
4,100+ Nobel laureates urge Greenpeace to stop o...


# Data pre-processing
## Lemmitize
In order to properly identify topics, we need to standardize the words in some way, removing extraneous information like tenses or plurals (e.g. "running", "runs" -> "run"). Lemmatization is the best way to do that, although it's not always the fastest way. In some cases, stemming will also work, althought it is not as precise.

Since we have a relatively small corpus to study, we will use lemmatization in order to be as precise as possible when cleaning up the words. To do this, we will feed in a `pos` parameter that tells our lemmatizing function to perform stemming with context.

Natural Language Toolkit's `stem` package includes a handy function called `WordNetLemmatizer` that makes lemmatizing our words simple. 

In [5]:
# Lemmatize the words keeping the context (stemming is "dumb" so we won't)
# However if we have a much larger corpus, we might consider stemming (as it is faster)
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v') # pos='v' means it peforms stemming with context

## Remove stopwords and words shorter than 3 chars
We also need to remove common words that don't add value ("a", "the", etc). These are called _stopwords_, and most NLP libraries have built-in methods for dealing with them. In `gensim`, the stopwords list can be called using `gensim.parsing.preprocessing.STOPWORDS`.

In [6]:
# Remove stopwords and words shorter than 3 characters, then lemmatize
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize(token))
    return result

## Check outputs
Let's compare the regular version of a sample headlind with the tokenized and lemmatized version. You'll note they are slightly different, but retain the same context.

In [7]:
sample = headlines['News'][2]

print('original document: ')
words = []
for word in sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(sample))

original document: 
['The', 'president', 'of', 'France', 'says', 'if', 'Brexit', 'won,', 'so', 'can', 'Donald', 'Trump']


 tokenized and lemmatized document: 
['president', 'france', 'say', 'brexit', 'donald', 'trump']


## Preprocess the headlines and save the results
Now we feed all the headlines into the preprocessing function, which lemmatizes and removes stopwords. 

In [8]:
cleaned_headlines = headlines['News'].map(preprocess)
cleaned_headlines[:5] # Check the results

0    [year, woman, mexico, city, finally, receive, ...
1      [chief, back, athens, permanent, olympic, host]
2      [president, france, say, brexit, donald, trump]
3    [british, police, hours, notice, threaten, hun...
4    [nobel, laureates, urge, greenpeace, stop, opp...
Name: News, dtype: object

## Count the word occurences using Bag of Words
We can count the words using different approaches. Bag of Words will simply count all remaining words left in our corpus.

In [9]:
# First we need to convert the words to numeric indices. We can do that with a gensim Dictionary.

# corpora.Dictionary implements the concept of a Dictionary – a mapping between words and their integer ids.
# https://radimrehurek.com/gensim/corpora/dictionary.html

dictionary = gensim.corpora.Dictionary(cleaned_headlines)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 alvarez
1 bear
2 birth
3 certificate
4 city
5 die
6 finally
7 hours
8 later
9 lira
10 mexico


In [10]:
'''
Filter out irrelevant words:
Keep tokens that appear in at least 15 documents
Keep only the 100,000 most frequent tokens
'''

dictionary.filter_extremes(no_below=15, keep_n=100000)

In [17]:
'''
Apply `doc2bow` to each headline, which createst a Bag of Words dictionary that reports how many words 
are in the headline and how many times each word appears.

Save this to an array, then check the example headline from above.
'''

bow_corpus = [dictionary.doc2bow(headline) for headline in cleaned_headlines]
print(bow_corpus[2])

bow_doc_2 = bow_corpus[2]

for i in range(len(bow_doc_2)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_2[i][0], 
                                            dictionary[bow_doc_2[i][0]], 
                                            bow_doc_2[i][1]))

[(21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]
Word 21 ("brexit") appears 1 time.
Word 22 ("donald") appears 1 time.
Word 23 ("france") appears 1 time.
Word 24 ("president") appears 1 time.
Word 25 ("say") appears 1 time.
Word 26 ("trump") appears 1 time.


## Count the word occurences using TF-IDF
We'll also test whether TF-IDF works better than Bag of Words for creating a corpus that, when fed into our LDA model, will produce coherent and accurate topics. 

In `gensim`, TF-IDF builds on Bag of Words, taking in processed BoW corpus data and returning a number that reflects how important a word is to each headline in the overall corpus. Conversely, the scikit-learn implementation of TF-IDF handles this all in one step.

In [21]:
from gensim import corpora, models

# Create the model
tfidf = models.TfidfModel(bow_corpus)

# Feed in the BoW corpus to the TF-IDF model
corpus_tfidf = tfidf[bow_corpus]

# Print the same headline from above
corpus_tfidf[2]

[(21, 0.5271930944921088),
 (22, 0.5289685708527566),
 (23, 0.3076906708092066),
 (24, 0.25211992539378436),
 (25, 0.18133987533447585),
 (26, 0.5011362450471095)]

## Running LDA using Bag of Words
We'll first run `LdaMulticore` using our `bow_corpus` and limiting our number of topics to 10. We're also going to print out the resulting topics and how the dominant words score in each topic. The parameter `id2word` helps us pass the numeric indices in our dictionary back into human-readable words.

In [28]:
# We have to play around with how many passes to take over the corpus in order to make the topics coherent
lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)

In [29]:
# Print each topic and how the words in those topics score (their relative weight)
for idx, topic in lda_model_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.012*"court" + 0.010*"police" + 0.008*"torture" + 0.008*"abuse" + 0.007*"german" + 0.006*"pirate" + 0.006*"say" + 0.006*"kill" + 0.006*"pope" + 0.006*"rule"
Topic: 1 
Words: 0.020*"drug" + 0.019*"kill" + 0.012*"mexico" + 0.010*"president" + 0.009*"year" + 0.007*"say" + 0.007*"afghanistan" + 0.007*"afghan" + 0.007*"zimbabwe" + 0.007*"mugabe"
Topic: 2 
Words: 0.021*"china" + 0.013*"years" + 0.010*"chinese" + 0.010*"kill" + 0.008*"pakistan" + 0.007*"media" + 0.007*"dead" + 0.007*"children" + 0.007*"internet" + 0.007*"attack"
Topic: 3 
Words: 0.016*"say" + 0.014*"attack" + 0.012*"russia" + 0.009*"government" + 0.009*"syria" + 0.009*"france" + 0.008*"iran" + 0.007*"president" + 0.007*"kill" + 0.007*"troop"
Topic: 4 
Words: 0.027*"israel" + 0.013*"right" + 0.012*"state" + 0.012*"israeli" + 0.011*"palestinian" + 0.009*"president" + 0.009*"human" + 0.009*"egypt" + 0.008*"palestinians" + 0.006*"egyptian"
Topic: 5 
Words: 0.030*"iran" + 0.022*"north" + 0.022*"korea" + 0.018*"so

## Running LDA using TF-IDF
We'll run the same LDA model on the corpus processed using TF-IDF, to see if the topics generated here are any more meaningful. We'll use the same other parameters in our `LdaMulticore` model, for consistency.

In [30]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.014*"korea" + 0.014*"nuclear" + 0.012*"north" + 0.011*"iran" + 0.007*"china" + 0.005*"missile" + 0.005*"ahmadinejad" + 0.005*"russia" + 0.005*"israel" + 0.005*"world"
Topic: 1 Word: 0.006*"sentence" + 0.005*"prison" + 0.005*"jail" + 0.005*"iran" + 0.004*"years" + 0.004*"death" + 0.004*"court" + 0.004*"internet" + 0.003*"mumbai" + 0.003*"torture"
Topic: 2 Word: 0.014*"gaza" + 0.011*"israel" + 0.007*"israeli" + 0.006*"drug" + 0.006*"mugabe" + 0.005*"chavez" + 0.005*"mexico" + 0.005*"police" + 0.005*"zimbabwe" + 0.004*"attack"
Topic: 3 Word: 0.004*"ossetia" + 0.004*"pirate" + 0.004*"minister" + 0.004*"police" + 0.004*"kill" + 0.003*"israel" + 0.003*"world" + 0.003*"mubarak" + 0.003*"dead" + 0.003*"attack"
Topic: 4 Word: 0.007*"china" + 0.006*"world" + 0.006*"olympics" + 0.006*"olympic" + 0.006*"protest" + 0.005*"russia" + 0.005*"iran" + 0.005*"bank" + 0.005*"putin" + 0.004*"billion"
Topic: 5 Word: 0.008*"georgia" + 0.006*"world" + 0.006*"food" + 0.004*"price" + 0.004*"ass

## Compare the performance of the two preprocessing steps in the LDA model

In [42]:
# our test headline from above
cleaned_headlines[2]

['president', 'france', 'say', 'brexit', 'donald', 'trump']

### Bag of Words LDA model scores

In [34]:
for index, score in sorted(lda_model_bow[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_bow.print_topic(index, 10)))


Score: 0.5333513021469116	 
Topic: 0.030*"iran" + 0.022*"north" + 0.022*"korea" + 0.018*"south" + 0.016*"russia" + 0.015*"nuclear" + 0.010*"china" + 0.009*"say" + 0.009*"georgia" + 0.009*"military"

Score: 0.35183143615722656	 
Topic: 0.016*"say" + 0.014*"attack" + 0.012*"russia" + 0.009*"government" + 0.009*"syria" + 0.009*"france" + 0.008*"iran" + 0.007*"president" + 0.007*"kill" + 0.007*"troop"

Score: 0.01435746531933546	 
Topic: 0.020*"drug" + 0.019*"kill" + 0.012*"mexico" + 0.010*"president" + 0.009*"year" + 0.007*"say" + 0.007*"afghanistan" + 0.007*"afghan" + 0.007*"zimbabwe" + 0.007*"mugabe"

Score: 0.014355541206896305	 
Topic: 0.023*"iraq" + 0.013*"death" + 0.013*"saudi" + 0.010*"women" + 0.009*"force" + 0.008*"home" + 0.008*"kill" + 0.007*"woman" + 0.007*"arabia" + 0.006*"muslim"

Score: 0.014352452009916306	 
Topic: 0.012*"court" + 0.010*"police" + 0.008*"torture" + 0.008*"abuse" + 0.007*"german" + 0.006*"pirate" + 0.006*"say" + 0.006*"kill" + 0.006*"pope" + 0.006*"rule"



### TF-IDF LDA model scores

In [35]:
for index, score in sorted(lda_model_tfidf[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index)))


Score: 0.8712913990020752	 
Topic: 0.014*"gaza" + 0.011*"israel" + 0.007*"israeli" + 0.006*"drug" + 0.006*"mugabe" + 0.005*"chavez" + 0.005*"mexico" + 0.005*"police" + 0.005*"zimbabwe" + 0.004*"attack"

Score: 0.01430324837565422	 
Topic: 0.006*"israel" + 0.005*"vote" + 0.004*"right" + 0.004*"iran" + 0.004*"riot" + 0.004*"say" + 0.004*"israeli" + 0.004*"state" + 0.004*"parliament" + 0.003*"party"

Score: 0.014301725663244724	 
Topic: 0.007*"china" + 0.006*"world" + 0.006*"olympics" + 0.006*"olympic" + 0.006*"protest" + 0.005*"russia" + 0.005*"iran" + 0.005*"bank" + 0.005*"putin" + 0.004*"billion"

Score: 0.014301386661827564	 
Topic: 0.014*"korea" + 0.014*"nuclear" + 0.012*"north" + 0.011*"iran" + 0.007*"china" + 0.005*"missile" + 0.005*"ahmadinejad" + 0.005*"russia" + 0.005*"israel" + 0.005*"world"

Score: 0.0143008166924119	 
Topic: 0.017*"kill" + 0.010*"pakistan" + 0.009*"afghanistan" + 0.009*"bomb" + 0.008*"troop" + 0.008*"iraq" + 0.008*"taliban" + 0.007*"attack" + 0.007*"afghan" 

Both approaches have relatively high accuracy with classification (> .80), and mostly make sense when reviewed by a human. What if we predicted topics for a headline that does not appear in our training set?

### Testing models on a new headline
When using an unseen headline, the Bag of Words-trained LDA model appears to classify the headline's topic more accurately.

* Bag of Words top topic: `Topic: 0.030*"iran" + 0.022*"north" + 0.022*"korea" + 0.018*"south" + 0.016*"russia"`
* TF-IDF top topic: `Topic: 0.014*"gaza" + 0.011*"israel" + 0.007*"israeli" + 0.006*"drug" + 0.006*"mugabe"`

In [40]:
unseen_document = 'Trump finally broke with Fauci'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

print("BoW approach\n")
for index, score in sorted(lda_model_bow[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_bow.print_topic(index, 5)))
    
print("\nTF-IDF approach")
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

BoW approach

Score: 0.7748793959617615	 Topic: 0.030*"iran" + 0.022*"north" + 0.022*"korea" + 0.018*"south" + 0.016*"russia"
Score: 0.0250187199562788	 Topic: 0.021*"china" + 0.013*"years" + 0.010*"chinese" + 0.010*"kill" + 0.008*"pakistan"
Score: 0.02501658909022808	 Topic: 0.023*"iraq" + 0.013*"death" + 0.013*"saudi" + 0.010*"women" + 0.009*"force"
Score: 0.025015806779265404	 Topic: 0.012*"court" + 0.010*"police" + 0.008*"torture" + 0.008*"abuse" + 0.007*"german"
Score: 0.025015080347657204	 Topic: 0.028*"gaza" + 0.022*"israel" + 0.018*"minister" + 0.013*"israeli" + 0.011*"say"
Score: 0.02501382865011692	 Topic: 0.018*"protest" + 0.017*"police" + 0.016*"government" + 0.009*"european" + 0.009*"people"
Score: 0.025012344121932983	 Topic: 0.026*"world" + 0.010*"japan" + 0.008*"nuclear" + 0.008*"power" + 0.008*"china"
Score: 0.02500961162149906	 Topic: 0.027*"israel" + 0.013*"right" + 0.012*"state" + 0.012*"israeli" + 0.011*"palestinian"
Score: 0.02500947378575802	 Topic: 0.020*"drug" 

## Export the model
We'll use the LDA-Bag of words model, which appears to be more accurate (based on our test documents).

In [41]:
import joblib
joblib.dump(lda_model_bow, "lda_bow.gz")

['lda_bow.gz']

## Append topics and scores to cleaned dataset

In [42]:
# This function returns the best-scoring topic and score assocated with a headline
def topic_getter(headline):
    if headline == np.nan:
        return [np.nan,np.nan]
    else:
        processed_headline = dictionary.doc2bow(preprocess(headline))
        topic_scores = sorted(lda_model_bow[processed_headline], key=lambda tup: -1*tup[1])
        topic = topic_scores[0][0]
        score = topic_scores[0][1]
        return [topic, score]

topic_getter(unseen_document)

[5, 0.77487856]

In [48]:
# Load dataset with combined data
all_data = pd.read_csv('../Data/Combined_News_DJIA.csv')
all_data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [49]:
all_data.dropna(how='any', inplace=True)

In [50]:
# Loop through the columns containing headlines and apply `topic_getter` to each headline, to retrieve topic
# Assign that data to a new column

for x in range(1,26): # for each column
    col = "Top" + str(x)
    new_col = col + "_topic_score"
    
    # for each row in column
    # get the headline, feed it into the lda_model_bow to get the topic and score
    all_data[new_col] = all_data[col].apply(topic_getter)

In [51]:
# Check the output
all_data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16_topic_score,Top17_topic_score,Top18_topic_score,Top19_topic_score,Top20_topic_score,Top21_topic_score,Top22_topic_score,Top23_topic_score,Top24_topic_score,Top25_topic_score
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,"[5, 0.6031984]","[2, 0.47149044]","[3, 0.36061332]","[5, 0.45405966]","[5, 0.5703015]","[5, 0.54791677]","[5, 0.49842814]","[1, 0.8499608]","[6, 0.43135294]","[1, 0.81994975]"
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,"[9, 0.53071535]","[9, 0.611838]","[7, 0.5506175]","[6, 0.81994605]","[0, 0.57922643]","[4, 0.6275869]","[4, 0.6561521]","[5, 0.51787627]","[6, 0.37684822]","[6, 0.7749493]"
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,"[5, 0.8499421]","[5, 0.6254238]","[8, 0.44398957]","[5, 0.8199832]","[1, 0.52361417]","[5, 0.7748322]","[5, 0.573867]","[5, 0.28484675]","[1, 0.69997424]","[6, 0.8499552]"
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,"[2, 0.69973093]","[5, 0.35203508]","[3, 0.48795778]","[3, 0.5979013]","[5, 0.46311668]","[5, 0.3666421]","[3, 0.5055619]","[8, 0.3956947]","[3, 0.71894366]","[3, 0.6346879]"
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,"[6, 0.5339442]","[5, 0.49618754]","[3, 0.43610215]","[5, 0.42683432]","[9, 0.5254859]","[8, 0.88747185]","[7, 0.574972]","[8, 0.4857589]","[2, 0.46074417]","[8, 0.6320944]"


In [52]:
# Save updated data
all_data.to_csv("../Data/Combined_News_DJIA_topics.csv")